In [1]:
from dash.dependencies import Input, Output
from plotly import graph_objs as go
from plotly.graph_objs import *
from datetime import datetime as dt

import dash
import dash_core_components as dcc
import dash_html_components as html

import pandas as pd
import os
import numpy as np
import h5py
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import shapefile
import descartes
from shapely.geometry import Point
import gmaps
import gmaps.datasets
import geopandas
import plotly.graph_objects as go # or plotly.express as px
import plotly.express as px
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import plotly.figure_factory as ff
import re
from datetime import datetime as dt
import json
import dash_bootstrap_components as dbc

from urllib.request import urlopen
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
     counties = json.load(response)

In [3]:
df = pd.read_csv('/Users/jiaqilu/Desktop/ENV19/data_visualization/GHC_sample_waterquality.csv')


/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:

type_input = {'county':['XCO2','XCH4'],
              'station':['Dissolved oxygen (DO)',"Orthophosphate",'pH','Chloride','Sulfate']}



external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}



#Side Panel  ---------------------------------------------------------------

# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 900,
    "bottom": 0,
    "width": "80rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}


sidebar = html.Div(
    [
        html.H2("Sidebar", className="display-4"),
        html.Hr(),
        html.P(
            "A simple sidebar layout with statistical graphs", className="lead"
        ),
        
        dcc.Graph(
        id='stats-graph'
        ),
        
        dcc.Graph(
        id='stats-2020-graph'
        ),
    
    ],
    style=SIDEBAR_STYLE,
    

)

content_sidebar = html.Div(id="page-content", style=CONTENT_STYLE)


#Dropdown menu ---------------------------------------------------------------

# separate station variables and county variables
# this list is not complete
variable_type = {
    'XCO2':'county',
    'XCH4':'county',
    'NO2':'station',
    'PM2.5':'county',
    'Dissolved Oxygen':'station',
    'Orthophosphate':'station',
    'COVID Cases':'county'
}
# unit for each variable
unitmap = {
    'XCO2':'ppm of ',
    'XCH4':'ppm of ',
    'NO2':'ppm of ',
    'PM2.5':'ppm of ', 
    'Dissolved Oxygen':'blah unit of ',
    'Orthophosphate':'blah unit of ',
    'COVID Cases':''
}

dropdown = html.Div([
    dcc.Dropdown(id="slct_var",
                 options=[
                     {"label": "Greenhouse Gas - XCO2", "value": "XCO2"},
                     {"label": "Greenhouse Gas - XCH4", "value": "XCH4"},
                     {"label": "Air Quality - NO2", "value": "NO2"},
                     {"label": "Air Quality - PM2.5", "value": "PM2.5"},
                     {"label": "Water Quality - Dissolved Oxygen", "value": 'Dissolved oxygen (DO)'},
                     {"label": "Water Quality - Orthophosphate", "value": "Orthophosphate"},
                     {"label": "Water Quality - pH", "value": "pH"},
                     {"label": "Water Quality - Sulfate", "value": "Sulfate"},
                     {"label": "Water Quality - Chloride", "value": "Chloride"},
                     {"label": "COVID Cases", "value": "COVID Cases"}
                 ],
                 multi=False,
                 value="XCH4",
                 style={'width': "40%"}
                 )
])

content_dropdown = html.Div(id="dropdown-content")

#APP LAYOUT---------------------------------------------------------------
app.layout = html.Div([
    html.H1("Texas Environmental Data and COVID19"),
    html.H4("Select Variable"),
    html.Div([dcc.Location(id="sel_var"), dropdown, content_dropdown]),
    html.H4("Select Date"),
    dcc.DatePickerSingle(
        id='my-date-picker-single',
        min_date_allowed=dt(2015, 1, 1),
        max_date_allowed=dt(2020, 7, 1),
        initial_visible_month=dt(2019, 6, 14),
        date=str(dt(2019, 6, 14))
    ),
    html.Div(id='output-container-date-picker-single'),
    
    html.Div([
        dcc.Graph(id='the_graph')
    ]),
    
    html.Div([
        dcc.Markdown("""
            **Click region**
            Click on regions on the graph.
        """),
        html.Pre(id='click-data', style=styles['pre']),
    ], className='three columns'),
    
    html.Div([dcc.Location(id="url"), sidebar, content_sidebar]),
    
    
    
])



#---------------------------------------------------------------


@app.callback(
    [Output('output-container-date-picker-single', 'children'),
    Output(component_id='the_graph', component_property='figure')
    ],
    [Input('my-date-picker-single', 'date'),Input('slct_var','value')])


def update_output(date, option_var):
    string_prefix = 'You have selected: '
    date_string = '2020-05-01'
    merged = df.loc[df['var'] == str(option_var),:]
    if  str(option_var) in type_input['county']:
        if date is not None:
            date = dt.strptime(re.split('T| ', date)[0], '%Y-%m-%d')
            date_string = date.strftime('%Y-%m-%d')
            temp = merged.loc[merged.date == date_string,:]
            print(merged.head())
            trace= go.Choroplethmapbox(geojson = counties,
                               locations = temp['fips'],
                               z = temp['value'],
                               colorscale='hot',
                               reversescale = True,
                               colorbar_thickness=20,
                               text = temp['county'],
                               marker_line_color='white',
                               customdata = temp['fips'],
                               hovertemplate = '<b>County</b>: <b>%{text}</b>'+
                                                 '<br> <b>Val </b>: %{z}<br>'+ '<extra></extra>' 
                            )
            fig= go.Figure(data = trace)
            mapboxt = 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2ozcGI1MTZ3MDBpcTJ3cXR4b3owdDQwaCJ9.8jpMunbKjdq1anXwU5gxIw'

            fig.update_layout(title_text= 'Texas Heatmap',
                              title_x=0.5, width = 700, height=700,
                              mapbox = dict(center= dict(lat=31.3915,  lon=-99.1707),
                                             accesstoken= mapboxt,style = 'basic',
                                             zoom=4.5,
                                           ),
                             clickmode='event+select'
                             );
    
    elif str(option_var) in type_input['station']:
        if date is not None:
            date = dt.strptime(re.split('T| ', date)[0], '%Y-%m-%d')
            date_string = date.strftime('%Y-%m-%d')
            
            temp = merged.loc[merged.date == date_string,:]
            
            print(temp)

            fig = px.scatter_mapbox(temp, lat="lat", lon="lon", hover_name = 'MonitoringLocationIdentifier', color = 'value', size = 
                                'value',
            color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)

            mapboxt = 'pk.eyJ1IjoiY2hyaWRkeXAiLCJhIjoiY2ozcGI1MTZ3MDBpcTJ3cXR4b3owdDQwaCJ9.8jpMunbKjdq1anXwU5gxIw'       

            fig.update_layout(title_text= 'Texas PointMap',
                              title_x=0.5, width = 700, height=700,
                              mapbox = dict(center= dict(lat=31.3915,  lon=-99.1707),
                                             accesstoken= mapboxt,style = 'basic',
                                             zoom=4.5,
                                           ));
    
    return (string_prefix + date_string, fig)

    
# Update side panel---------------------------------------------------------------    
@app.callback(
    [Output('click-data', 'children'),Output(component_id = 'stats-graph', component_property='figure'),
     Output(component_id = 'stats-2020-graph', component_property='figure')
    ],
    [Input('the_graph', 'clickData'), Input('slct_var','value'),Input('my-date-picker-single', 'date')])
def display_click_data(clickData, option_var, date):
    string = json.dumps(clickData)
    dictionary = json.loads(string)
    if str(option_var) in type_input['county']:
        if dictionary and 'points' in dictionary.keys():
            point = dictionary['points'][0]
            fips = int(point['location'])
            merged = df.loc[df['var'] == str(option_var),:]
            select =  merged.loc[merged['fips'] == fips,:]
            select = select.sort_values(by=['date'])
            select['date'] = pd.to_datetime(select['date'])
            county = str(select['county'].iloc[1])

            fig = px.line(select, x="date", y="value", title='Concentration of'+str(option_var)+' at '+ county +' county')

            fig.update_layout(xaxis_title='Date',
                       yaxis_title='CH4 Concentration (nmol/mol)')

            fig.update_traces(marker_size=20)
            fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                    buttons=list([
                        dict(count=1, label="1m", step="month", stepmode="backward"),
                        dict(count=6, label="6m", step="month", stepmode="backward"),
                        dict(count=1, label="YTD", step="year", stepmode="todate"),
                        dict(count=1, label="1y", step="year", stepmode="backward"),
                        dict(step="all")
                    ])
                )
            )

            select2020 = select.loc[select['date'].dt.year == 2020,:]
            
            fig = px.line(select2020, x="date", y="value", title='Concentration of'+str(option_var)+' at '+ county +' county')

            fig.update_layout(xaxis_title='Date',
                       yaxis_title='CH4 Concentration (nmol/mol)')

            fig.update_traces(marker_size=20)
            fig.update_xaxes(
                rangeslider_visible=True,
                rangeselector=dict(
                    buttons=list([
                        dict(count=1, label="1m", step="month", stepmode="backward"),
                        dict(count=6, label="6m", step="month", stepmode="backward"),
                        dict(count=1, label="YTD", step="year", stepmode="todate"),
                        dict(count=1, label="1y", step="year", stepmode="backward"),
                        dict(step="all")
                    ])
                )
            )
            
            fig.update_layout(
                shapes=[
                    # 1st highlight during April 4 - May 31
                    dict(
                        type="rect",
                        # x-reference is assigned to the x-values
                        xref="x",
                        # y-reference is assigned to the plot paper [0,1]
                        yref="paper",
                        x0="2015-02-04",
                        y0=0,
                        x1="2015-02-06",
                        y1=1,
                        fillcolor="LightSalmon",
                        opacity=0.5,
                        layer="below",
                        line_width=0,
                    )
                ]
            )
            
            
            
            return (fips,fig)
            
        else:
            raise Exception("POST /_dash-update-component HTTP/1.1")
            
#     elif str(option_var) in type_input['station']:
#         if dictionary and 'points' in dictionary.keys():
#             point = dictionary['points'][0]
#             loc = point['hovertext']
            
#             select =  df.loc[df['MonitoringLocationIdentifier'] == loc,:]
#             select = select.loc[select['date'] == date,: ]
#             select = select.drop_duplicates(subset=['var', 'lat', 'lon', 'MonitoringLocationIdentifier','value','date'],keep = 'first')
            
            
#             date_str = str(date).split()[0]
            
#             fig = go.Figure()
#             temp = select[select['unit'].str.contains("mg/l")]
#             fig = px.bar(temp, y='value', x='var', text='value')
#             fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
#             fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
#             fig.update_layout(xaxis_title='Variable',
#                               yaxis_title='Concentration (mg/l)',
#                               title='Concentration of water pollutants at '+ str(loc) +' on '+ date_str)
#             return (loc,fig)
            
#         else:
#             raise Exception("POST /_dash-update-component HTTP/1.1")

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Jul/2020 15:58:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2020 15:58:28] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2020 15:58:28] "GET /_dash-dependencies HTTP/1.1" 200 -


            date        value   county     fips      unit   var    year  lat  \
9668  2015-01-01  1776.963472   Oldham  48359.0  nmol/mol  XCH4  2015.0  NaN   
9669  2015-01-01  1728.534698   Reeves  48389.0  nmol/mol  XCH4  2015.0  NaN   
9670  2015-01-01  1770.444214   Crosby  48107.0  nmol/mol  XCH4  2015.0  NaN   
9671  2015-01-01  1750.886440  Andrews  48003.0  nmol/mol  XCH4  2015.0  NaN   
9672  2015-01-01  1774.169505   Carson  48065.0  nmol/mol  XCH4  2015.0  NaN   

      lon MonitoringLocationIdentifier  
9668  NaN                          NaN  
9669  NaN                          NaN  
9670  NaN                          NaN  
9671  NaN                          NaN  
9672  NaN                          NaN  
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/jiaqilu/opt/anaconda3/lib/python

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/flask/app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "/Users/jiaqilu/

127.0.0.1 - - [22/Jul/2020 15:58:30] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [22/Jul/2020 15:58:32] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [22/Jul/2020 15:58:33] "POST /_dash-update-component HTTP/1.1" 200 -


            date        value  county     fips      unit   var    year  lat  \
80013 2020-01-01  1826.323569  Reeves  48389.0  nmol/mol  XCH4  2020.0  NaN   
80174 2020-01-07  1829.117537  Reeves  48389.0  nmol/mol  XCH4  2020.0  NaN   
80219 2020-01-08  1841.224730  Reeves  48389.0  nmol/mol  XCH4  2020.0  NaN   
80311 2020-01-10  1806.765795  Reeves  48389.0  nmol/mol  XCH4  2020.0  NaN   
80464 2020-01-14  1827.254891  Reeves  48389.0  nmol/mol  XCH4  2020.0  NaN   

       lon MonitoringLocationIdentifier  
80013  NaN                          NaN  
80174  NaN                          NaN  
80219  NaN                          NaN  
80311  NaN                          NaN  
80464  NaN                          NaN  
